In [ ]:
import os

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import shap
import tqdm.notebook as tqdm
from cuml.ensemble import RandomForestClassifier
from mordred import Calculator, descriptors
from rdkit import Chem
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import average_precision_score, \
    balanced_accuracy_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, learning_curve, \
    StratifiedShuffleSplit
from sklearn.pipeline import Pipeline

from correlation_threshold import CorrelationThreshold

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='sans-serif')
sns.set_palette(['#6da7de', '#9e0059', '#dee000', '#d82222', '#5ea15d',
                 '#943fa6', '#63c5b5', '#ff38ba', '#eb861e', '#ee266d'])
sns.set_context('paper', font_scale=1.3)

## Molecular descriptor feature generation

In [ ]:
compounds = pd.read_csv('../data/compound_smiles.csv')
mols = compounds['SMILES (Canonical)'].apply(Chem.MolFromSmiles)
clss = compounds['Skin'].astype(np.int32)

In [ ]:
# Calculate features using Mordred.
mordred_calculator = Calculator(descriptors, ignore_3D=True)
# Exclude features that contain non-numeric values.
features = pd.DataFrame(mordred_calculator.pandas(mols)
                        .select_dtypes(exclude='object')
                        .astype(np.float32))

## Random forest training

In [ ]:
n_splits = 100
test_size = 0.2

variance_threshold = 0.05
correlation_threshold = 0.95
n_trees = 1000

In [ ]:
# Repeatedly evaluate the model to estimate its performance.
accuracies_train = np.zeros(n_splits, np.float32)
accuracies_test = np.zeros(n_splits, np.float32)
average_precisions_train = np.zeros(n_splits, np.float32)
average_precisions_test = np.zeros(n_splits, np.float32)
roc_aucs_train = np.zeros(n_splits, np.float32)
roc_aucs_test = np.zeros(n_splits, np.float32)
interval = np.linspace(0, 1, 101, dtype=np.float32)
tprs_train = np.zeros((n_splits, 101), np.float32)
tprs_test = np.zeros((n_splits, 101), np.float32)
precisions_train = np.zeros((n_splits, 101), np.float32)
precisions_test = np.zeros((n_splits, 101), np.float32)
best_max_depth = np.zeros(n_splits, np.uint8)
for i, (train_index, test_index) in tqdm.tqdm(
        enumerate(
            StratifiedShuffleSplit(
                n_splits, test_size=test_size, random_state=42).split(
                features.values, clss)),
        desc='Cross-validation', total=n_splits):
    features_train = features.values[train_index]
    features_test = features.values[test_index]
    clss_train, clss_test = clss[train_index], clss[test_index]
    
    grid_search = GridSearchCV(
        Pipeline([
            ('variance_threshold', VarianceThreshold(variance_threshold)),
            ('correlation_threshold', CorrelationThreshold(
                correlation_threshold)),
            ('classify', RandomForestClassifier(
                n_estimators=n_trees, random_state=42))]),
        param_grid={'classify__max_depth': np.arange(3, 9, 1)},
        cv=StratifiedShuffleSplit(n_splits, test_size=test_size,
                                  random_state=42))
    
    grid_search.fit(features_train, clss_train)
    pred_train = grid_search.predict_proba(features_train)[:, 1]
    pred_test = grid_search.predict_proba(features_test)[:, 1]
    
    # Compute evaluation metrics on the train and test data.
    accuracies_train[i] = balanced_accuracy_score(
            clss_train, np.asarray(pred_train > 0.5, np.int))
    accuracies_test[i] = balanced_accuracy_score(
            clss_test, np.asarray(pred_test > 0.5, np.int))
    average_precisions_train[i] = average_precision_score(
        clss_train, pred_train)
    average_precisions_test[i] = average_precision_score(
        clss_test, pred_test)
    roc_aucs_train[i] = roc_auc_score(clss_train, pred_train)
    roc_aucs_test[i] = roc_auc_score(clss_test, pred_test)
    fpr_train, tpr_train, _ = roc_curve(clss_train, pred_train)
    tprs_train[i] = np.interp(interval, fpr_train, tpr_train)
    fpr_test, tpr_test, _ = roc_curve(clss_test, pred_test)
    tprs_test[i] = np.interp(interval, fpr_test, tpr_test)
    precision_train, recall_train, _ = precision_recall_curve(
        clss_train, pred_train)
    precisions_train[i] = np.interp(
        interval, recall_train[::-1], precision_train[::-1])
    precision_test, recall_test, _ = precision_recall_curve(
        clss_test, pred_test)
    precisions_test[i] = np.interp(
        interval, recall_test[::-1], precision_test[::-1])
    
    # Save optimal hyperparameter(s).
    best_max_depth[i] = (grid_search.best_estimator_
                         .named_steps['classify'].max_depth)

In [ ]:
stats = {'accuracy_train': np.mean(accuracies_train),
         'accuracy_std_train': np.std(accuracies_train),
         'average_precision_train': np.mean(average_precisions_train),
         'average_precision_std_train': np.std(average_precisions_train),
         'roc_auc_train': np.mean(roc_aucs_train),
         'roc_auc_std_train': np.std(roc_aucs_train),
         'tpr_mean_train': np.mean(tprs_train, axis=0),
         'tpr_std_train': np.std(tprs_train, axis=0),
         'precision_mean_train': np.mean(precisions_train, axis=0),
         'precision_std_train': np.std(precisions_train, axis=0),
        
         'accuracy_test': np.mean(accuracies_test),
         'accuracy_std_test': np.std(accuracies_test),
         'average_precision_test': np.mean(average_precisions_test),
         'average_precision_std_test': np.std(average_precisions_test),
         'roc_auc_test': np.mean(roc_aucs_test),
         'roc_auc_std_test': np.std(roc_aucs_test),
         'tpr_mean_test': np.mean(tprs_test, axis=0),
         'tpr_std_test': np.std(tprs_test, axis=0),
         'precision_mean_test': np.mean(precisions_test, axis=0),
         'precision_std_test': np.std(precisions_test, axis=0),
        
         'max_depth': scipy.stats.mode(best_max_depth)[0][0]}

In [ ]:
print(f'Accuracy: {stats["accuracy_test"]:.3f} ± '
      f'{stats["accuracy_std_test"]:.3f}')
print(f'AUROC: {stats["roc_auc_test"]:.3f} ± '
      f'{stats["roc_auc_std_test"]:.3f}')
print(f'Average precision: {stats["average_precision_test"]:.3f} ± '
      f'{stats["average_precision_std_test"]:.3f}')
print(f'Optimal number of maximum depth (mode): {stats["max_depth"]}')

In [ ]:
# Train the optimal model on the entire dataset and export.
classifier = Pipeline([
    ('variance_threshold', VarianceThreshold(variance_threshold)),
    ('correlation_threshold', CorrelationThreshold(correlation_threshold)),
    ('classify', RandomForestClassifier(n_estimators=n_trees,
                                        max_depth=stats['max_depth'],
                                        random_state=42))])
classifier.fit(features.values, clss)
_ = joblib.dump(classifier, 'rf.joblib')

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

interval = np.linspace(0, 1, 101)
tpr_test = stats['tpr_mean_test']
tpr_test[0], tpr_test[-1] = 0, 1
ax.plot(interval, tpr_test,
        label=f'AUC = {stats["roc_auc_test"]:.3f} '
              f'± {stats["roc_auc_std_test"]:.3f}')
ax.fill_between(interval, tpr_test - stats['tpr_std_test'],
                tpr_test + stats['tpr_std_test'], alpha=0.2)
        
ax.plot([0, 1], [0, 1], c='black', ls='--')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('roc.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))
        
precision_test = stats['precision_mean_test']
ax.plot(interval, precision_test,
        label=f'Avg precision = '
              f'{stats["average_precision_test"]:.3f} ± '
              f'{stats["average_precision_std_test"]:.3f}')
ax.fill_between(interval, precision_test - stats['precision_std_test'],
                precision_test + stats['precision_std_test'], alpha=0.2)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('pr.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

### Learning curve

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    classifier, features.values, clss, train_sizes=np.linspace(0.2, 1.0, 10),
    cv=StratifiedShuffleSplit(n_splits, test_size=0.2, random_state=42),
    scoring='roc_auc')

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
ax.plot(train_sizes, test_scores_mean, 'o-', label='Test score')
ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                test_scores_mean + test_scores_std, alpha=0.2)
ax.plot(train_sizes, train_scores_mean, 'o-', label='Training score')
ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                train_scores_mean + train_scores_std, alpha=0.2)

ax.set_xlabel('Number of training examples')
ax.set_ylabel('AUROC')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## SHAP feature importances

In [ ]:
# Feature importances.
predict_proba = lambda x: classifier.predict_proba(x)[:,1]
features_median = features.median().values.reshape((1, -1))
explainer = shap.KernelExplainer(
    predict_proba, shap.kmeans(features.values, 50), link='logit')

In [ ]:
shap_values = explainer.shap_values(features)

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

importances = np.abs(np.mean(shap_values, axis=0))
order = np.argsort(importances)[::-1]
sns.barplot(x=np.arange(20), y=importances[order][:20], color='#6da7de')

ax.set_xticklabels(features.columns.values[order][:20], rotation=90)
ax.set_ylabel('SHAP feature importance')

sns.despine()

plt.savefig('feature_importance_global.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

column_i = features.columns.get_loc('MW')
ax.scatter(features['MW'], shap_values[:, column_i])

ax.set_xlabel('Molecular weight')
ax.set_ylabel('SHAP value')

sns.despine()

plt.savefig('feature_importance_mw.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
test_compounds = pd.DataFrame({
    'compound': ['diphenhydramine', 'diphenhydramine N-hexose'],
    'smiles': ['CN(CCOC(c1ccccc1)c1ccccc1)C',
               'C[N+](CCOC(C1=CC=CC=C1)C2=CC=CC=C2)(OC3OC(CO)C(O)C(O)C3O)C']})
test_features = pd.DataFrame(
    mordred_calculator.pandas(test_compounds['smiles']
                              .apply(Chem.MolFromSmiles))
    [features.columns].astype(np.float32))
test_pred = classifier.predict_proba(test_features.values)[:, 1]
test_shap = explainer.shap_values(test_features)

for i, compound in enumerate(test_compounds['compound']):
    print(f'Score for {compound}: {test_pred[i]:.2f}')
    shap.plots.force(explainer.expected_value, test_shap[i],
                     test_features.iloc[i].round(2).astype(str),
                     matplotlib=True, show=False,
                     contribution_threshold=0.02)
    plt.savefig(f'shap_{compound}.png', dpi=300, bbox_inches='tight')
    plt.show()

## Export

In [ ]:
_ = joblib.dump(
    (stats, (train_sizes, train_scores, test_scores), shap_values),
    'drugs_epidermis.joblib')